In [1]:
import random

NUM = 100 # 每次生成100道题
TYPE_DICT = ['mul', 'div']

MUL1_MIN = 20
MUL1_MAX = 300
MUL2_MIN = 4
MUL2_MAX = 9

DIV1_MIN = 100
DIV1_MAX = 1199
DIV2_MIN = 3
DIV2_MAX = 9

# md_rnd: 乘除随机的计算
# five: 和5、10相关的计算
# squre: 平方和平方数计算
PROB_TYPE = ['md_rnd','five','squre']

NULL_4 = '\u0000' * 4
NULL_8 = '\u0000' * 8


In [2]:
type_rnd = random.randint(0, len(TYPE_DICT))
mul1 = random.randint(MUL1_MIN, MUL1_MAX)
mul2 = random.randint(MUL2_MIN, MUL2_MAX)

mul_res = mul1 * mul2

div1 = random.randint(DIV1_MIN, DIV1_MAX)
div2 = random.randint(DIV2_MIN, DIV2_MAX)

div_res = div1//div2

div_rem = div1 % div2

print(f"mul1 = {mul1}, mul2 = {mul2} mul_res = {mul_res}")
print(f"div1 = {div1}, div2 = {div2} div_res = {div_res} div_rem ={div_rem}")


mul1 = 128, mul2 = 9 mul_res = 1152
div1 = 362, div2 = 3 div_res = 120 div_rem =2


In [3]:
def mul_problem_gen (index:int, mul1:int, mul2:int, debug: bool = False):
    mul_res = mul1 * mul2
    format_str = f"{index:<4} | {mul1:<4} x {mul2:>4} = {NULL_4} | {NULL_4} 答案：{mul_res}"
    if debug: print(format_str)
    return format_str
    

In [4]:
mul_problem_gen(1,18,3, True)
mul_problem_gen(122,55,5, True)

1    | 18   x    3 =      |      答案：54
122  | 55   x    5 =      |      答案：275


'122  | 55   x    5 = \x00\x00\x00\x00 | \x00\x00\x00\x00 答案：275'

In [5]:
def div_problem_gen (index:int, div1:int, div2:int,  debug: bool = False):

    div_res = div1//div2

    div_rem = div1 % div2
    
    ans_str = f"{div_res}" if div_rem == 0 else f"{div_res}...{div_rem}" 
    
    format_str = f"{index:<4} | {div1:<4} \u00F7 {div2:>4} = {NULL_4} | {NULL_4} 答案：{ans_str}"
    if debug: print(format_str)
    return format_str

In [6]:
div_problem_gen(1,18,3, True)
div_problem_gen(12,55,15, True)

1    | 18   ÷    3 =      |      答案：6
12   | 55   ÷   15 =      |      答案：3...10


'12   | 55   ÷   15 = \x00\x00\x00\x00 | \x00\x00\x00\x00 答案：3...10'

In [7]:
def problem_gen (type_: str, index: int):
    problem_str = None
    if type_ == 'mul':
        mul1 = random.randint(MUL1_MIN, MUL1_MAX)
        mul2 = random.randint(MUL2_MIN, MUL2_MAX)
        problem_str = mul_problem_gen(index, mul1, mul2)
    elif type_ == 'div':
        div1 = random.randint(DIV1_MIN, DIV1_MAX)
        div2 = random.randint(DIV2_MIN, DIV2_MAX)
        problem_str = div_problem_gen(index, div1, div2)
    return problem_str
        

In [8]:
def problem_gen_five (type_: str, index: int):
    problem_str = None
    if type_ == 'mul':
        mul1 = random.randint(2, 19)*50
        mul2 = random.randint(2, 9)*20
        problem_str = mul_problem_gen(index, mul1, mul2)
    elif type_ == 'div':
        mul1 = random.randint(1, 19)
        mul2 = random.randint(1, 9)*10
        remain = random.randint(0, mul2)
        five_scale = random.randint(1, 9)*5
        product = (mul1 * mul2 + remain)*five_scale
        problem_str = div_problem_gen(index, product, mul2)
    return problem_str

In [9]:
def problem_gen_squre (type_: str, index: int):
    problem_str = None
    mul = random.randint(9, 20)
    product = mul * mul
    if type_ == 'mul':
        problem_str = mul_problem_gen(index, mul, mul)
    elif type_ == 'div':
        problem_str = div_problem_gen(index, product, mul)
    return problem_str

In [10]:
def problem_gen_mix(algo_type_: str, index: int, problem_type:str):
    if problem_type == 'md_rnd':
        print_str = problem_gen(algo_type_, index)
    elif problem_type == 'five':
        print_str = problem_gen_five(algo_type_, index)
    elif problem_type == 'squre':
        print_str = problem_gen_squre(algo_type_, index)
    elif problem_type == 'mix':
        single_problem_type = PROB_TYPE[random.randint(1, len(PROB_TYPE))-1]
        print_str = problem_gen_mix(algo_type_, index, single_problem_type)
    return print_str


In [11]:
def biased_random(threshold):
    '''随机选择是乘法或是除法, 基于阈值决定多少 50随机，0-49 乘法多一些，51-99 除法多一些。'''
    # 将阈值范围0-99映射到0.0-1.0的概率范围
    probability_of_one = (threshold / 100.0)
    # 生成一个0到1之间的随机数
    random_number = random.random()
    # 如果随机数小于等于阈值对应的1的概率，则输出1，否则输出0
    if random_number <= probability_of_one:
        return 1
    else:
        return 0

In [12]:
def print_problem_single(round_:int, special_type:str='md_rnd', threshold = 50):
    all_out = ''
    for i in range (round_):
        # 随机选择是乘法或是除法
        # algo_type_ = TYPE_DICT[random.randint(0, len(TYPE_DICT)-1)]
        algo_type_ = TYPE_DICT[biased_random(threshold)]
        if special_type == 'md_rnd':
            print_str = problem_gen(algo_type_, i+1)
        elif special_type == 'five':
            print_str = problem_gen_five(algo_type_, i+1)
        elif special_type == 'squre':
            print_str = problem_gen_squre(algo_type_, i+1)
        elif special_type == 'mix':
            print_str = problem_gen_mix(algo_type_, i+1, special_type)

        out = f"{print_str:<50}\n"
        
        print(out)

        all_out += out

    return all_out   


In [13]:
def gen_problem_per_day(nums:int, special_type:str='md_rnd', bias:int=50):
    out=''
    out+=f"日期_____， 正确数目_____, 错误数目_____, 得分______, 类型:{special_type}\n"
    out+=print_problem_single(nums, special_type, bias)
    out+="\n"*2
    return out

In [14]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdf_name = "multi_page_problem.pdf"

# have to use SimSun, otherwise can not generate pdf correctly
font_name = 'SimSun'
font_path = './SimSun.ttf'

def print_problem_pdf(days, nums):
    pdfmetrics.registerFont(TTFont(font_name, font_path))  #注册字体
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(fontName=font_name, name='Sun', leading=15, fontSize=13))  #leading=spacing
      
    doc = SimpleDocTemplate(pdf_name, pagesize=letter)
    
    styles.add(ParagraphStyle(name='Address', fontName='Arial'))
    flowables = []

    for d in range(days):
        # 将换行符替换为LineBreak对象
        problem_text = gen_problem_per_day(nums, 'five', 70)
        lines = problem_text.split('\n')
        for line in lines:
            flowables.append(Paragraph(line, styles['Song']))

        # 添加额外的间距
        flowables.append(Spacer(1, 12))

        if d < days - 1:  # 除了最后一页外，其他每页末尾添加分页符
            flowables.append(PageBreak())

    doc.build(flowables)

# 使用函数生成多页PDF
days = 4  # 
nums = 40  # 
print_problem_pdf(days, nums)

1    | 3535 ÷   10 =      |      答案：353...5       

2    | 450  x  180 =      |      答案：81000         

3    | 900  x  140 =      |      答案：126000        

4    | 4525 ÷   20 =      |      答案：226...5       

5    | 6000 ÷   10 =      |      答案：600           

6    | 13380 ÷   70 =      |      答案：191...10     

7    | 900  x   40 =      |      答案：36000         

8    | 17730 ÷   90 =      |      答案：197          

9    | 250  x   40 =      |      答案：10000         

10   | 600  x  100 =      |      答案：60000         

11   | 6230 ÷   10 =      |      答案：623           

12   | 3725 ÷   50 =      |      答案：74...25       

13   | 36880 ÷   50 =      |      答案：737...30     

14   | 4000 ÷   50 =      |      答案：80            

15   | 2440 ÷   50 =      |      答案：48...40       

16   | 15580 ÷   40 =      |      答案：389...20     

17   | 9750 ÷   70 =      |      答案：139...20      

18   | 11600 ÷   90 =      |      答案：128...80     

19   | 13550 ÷   80 =      |      答案：169...30     

20   | 24150

KeyError: "Style 'Song' not found in stylesheet"